In [8]:
import pandas as pd

# Load the dataset (replace with your dataset file)
data = pd.read_csv("mypersonality_final.csv", encoding="latin1")

# Remove the first set of personality columns
data = data.drop(columns=["sEXT", "sNEU", "sAGR", "sCON", "sOPN"])

# Remove the additional columns
columns_to_remove = ["#AUTHID","NETWORKSIZE", "BETWEENNESS", "NBETWEENNESS", "DENSITY", "BROKERAGE", "NBROKERAGE", "TRANSITIVITY"]
data.drop(columns=columns_to_remove, inplace=True)

# Replace 'n' with 0 and 'y' with 1 for the remaining columns
columns_to_replace = ["cEXT", "cNEU", "cAGR", "cCON", "cOPN"]
data[columns_to_replace] = data[columns_to_replace].replace({'n': 0, 'y': 1})

# Remove the "c" from the columns "sEXT," "sNEU," "sAGR," "sCON," and "sOPN"
data.columns = data.columns.str.replace('c', '')

# Remove the "DATE" column
data = data.drop(columns=["DATE"])

# Print the modified dataset
print(data.head())

data.head(15)



                                              STATUS  EXT  NEU  AGR  CON  OPN
0                        likes the sound of thunder.    0    1    0    0    1
1  is so sleepy it's not even funny that's she ca...    0    1    0    0    1
2  is sore and wants the knot of muscles at the b...    0    1    0    0    1
3         likes how the day sounds in this new song.    0    1    0    0    1
4                                        is home. <3    0    1    0    0    1


,STATUS,EXT,NEU,AGR,CON,OPN
0,likes the sound of thunder.,0,1,0,0,1
1,is so sleepy it's not even funny that's she ca...,0,1,0,0,1
2,is sore and wants the knot of muscles at the b...,0,1,0,0,1
3,likes how the day sounds in this new song.,0,1,0,0,1
4,is home. <3,0,1,0,0,1
5,www.thejokerblogs.com,0,1,0,0,1
6,"saw a nun zombie, and liked it. Also, *PROPNAM...",0,1,0,0,1
7,is in Kentucky. 421 miles into her 1100 mile j...,0,1,0,0,1
8,was about to finish a digital painting before ...,0,1,0,0,1
9,is celebrating her new haircut by listening to...,0,1,0,0,1


In [23]:
data.shape

(9917, 6)

In [10]:
#I have imported the necessary libraries and packages, including Pandas for data handling, 
#Transformers for BERT integration, TensorFlow and Keras for deep learning, and other utilities.
import pandas as pd
from transformers import TFBertModel, BertTokenizer
import os
import re
import random
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import auc, roc_curve
import matplotlib.pyplot as plt

In [11]:
# Set a seed value for reproducibility
seed_value = 29
os.environ['PYTHONHASHSEED'] = str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

In [12]:
# Check for GPU availability
#device_name = tf.test.gpu_device_name()
#if device_name != '/device:GPU:0':
#    raise SystemError('GPU device not found')
#print('Found GPU at: {}'.format(device_name))

SystemError: GPU device not found

In [13]:
# The number of personality axes and the max sequance lenght was set for the BERT tokenization. 
N_AXIS = 5  # You have 5 axes: EXT, NEU, AGR, CON, and OPN
MAX_SEQ_LEN = 128
BERT_NAME = 'bert-base-uncased'#base is a smaller and hence a computationally efficient version of BERT 
# compared to 'large' variants; 'uncased'means that the model is trained on text where all the words are in lowercase. 

In [18]:
axes = ["EXT", "NEU", "AGR", "CON", "OPN"]

In [19]:
# Define the text preprocessing function for eliminating square brackets, links, numbers, and emojis etc
def text_preprocessing(text):
    text = text.lower()#converts all characters in the input text to lowercase
    text = re.sub('\[.*?\]', '', text)#remove text enclosed in square brackets
    text = re.sub('https?://\S+|www\.\S+', '', text)#removes websites 
    text = re.sub('<.*?>+', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text.encode('ascii', 'ignore').decode('ascii')
    if text.startswith("'"):
        text = text[1:-1]
    return text

In [20]:
data = data.sample(frac=1)  # Shuffle the data

In [21]:
# Extract the "STATUS" column and personality traits
sentences = data["STATUS"]
personality_traits = ["EXT", "NEU", "AGR", "CON", "OPN"]
labels = data[personality_traits]

In [24]:
# Define the ratios for train, validation, and test splits (adjust these values as needed)
train_ratio = 0.70  # 70% for training
val_ratio = 0.15   # 15% for validation
test_ratio = 0.15  # 15% for testing

In [25]:
total_samples = len(data)
train_samples = int(train_ratio * total_samples)
val_samples = int(val_ratio * total_samples)
test_samples = int(test_ratio * total_samples)

In [26]:
# Split your data into train, validation, and test sets
train_sentences = sentences[:train_samples]
y_train = labels[:train_samples]
val_sentences = sentences[train_samples:train_samples + val_samples]
y_val = labels[train_samples:train_samples + val_samples]
test_sentences = sentences[train_samples + val_samples:train_samples + val_samples + test_samples]
y_test = labels[train_samples + val_samples:train_samples + val_samples + test_samples]

In [27]:
# Define a function to prepare BERT input
def prepare_bert_input(sentences, seq_len, bert_name):
    tokenizer = BertTokenizer.from_pretrained(bert_name)
    encodings = tokenizer(sentences.tolist(), truncation=True, padding='max_length',
                          max_length=seq_len)
    input = [np.array(encodings["input_ids"]), np.array(encodings["token_type_ids"]),
             np.array(encodings["attention_mask"])]
    return input

In [28]:
X_train = prepare_bert_input(train_sentences, MAX_SEQ_LEN, BERT_NAME)
X_val = prepare_bert_input(val_sentences, MAX_SEQ_LEN, BERT_NAME)
X_test = prepare_bert_input(test_sentences, MAX_SEQ_LEN, BERT_NAME)

In [29]:
# Define the model architecture
input_ids = layers.Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32, name='input_ids')
input_type = layers.Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32, name='token_type_ids')
input_mask = layers.Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32, name='attention_mask')
inputs = [input_ids, input_type, input_mask]
bert = TFBertModel.from_pretrained(BERT_NAME)
bert_outputs = bert(inputs)
last_hidden_states = bert_outputs.last_hidden_state
avg = layers.GlobalAveragePooling1D()(last_hidden_states)
output = layers.Dense(N_AXIS, activation="sigmoid")(avg)
model = keras.Model(inputs=inputs, outputs=output)
model.summary()

C:\Users\swast\anaconda3\lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\swast\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.p

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 128)]        0           []                               
                                                                                                  
 token_type_ids (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  109482240   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'token_type_ids[0][0]',     

In [31]:
# Load the BERT model and set its layers as non-trainable
bert = TFBertModel.from_pretrained(BERT_NAME)
bert.trainable = False  # Set BERT layers as non-trainable

# Define the rest of the model architecture
bert_outputs = bert(inputs)
last_hidden_states = bert_outputs.last_hidden_state
avg = layers.GlobalAveragePooling1D()(last_hidden_states)
output = layers.Dense(N_AXIS, activation="sigmoid")(avg)
model = keras.Model(inputs=inputs, outputs=output)
model.summary()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 128)]        0           []                               
                                                                                                  
 token_type_ids (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 tf_bert_model_1 (TFBertModel)  TFBaseModelOutputWi  109482240   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'token_type_ids[0][0]',   

In [ ]:
# End-to-end fine-tuning
# Define optimizer, loss, and compile the model
max_epochs = 7
batch_size = 32
opt = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.BinaryCrossentropy()
best_weights_file = "weights.h5"
auc = tf.keras.metrics.AUC(multi_label=True, curve="ROC")
m_ckpt = ModelCheckpoint(best_weights_file, monitor='val_'+auc.name, mode='max', verbose=2,
                          save_weights_only=True, save_best_only=True)
model.compile(loss=loss, optimizer=opt, metrics=[auc, tf.keras.metrics.BinaryAccuracy()])

# Training code remains the same
model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=max_epochs,
    batch_size=batch_size,
    callbacks=[m_ckpt],
    verbose=2
)

Epoch 1/7

Epoch 1: val_auc_2 improved from -inf to 0.48536, saving model to weights.h5
217/217 - 1365s - loss: 0.6891 - auc_2: 0.4957 - binary_accuracy: 0.5490 - val_loss: 0.6754 - val_auc_2: 0.4854 - val_binary_accuracy: 0.5847 - 1365s/epoch - 6s/step
Epoch 2/7

Epoch 2: val_auc_2 improved from 0.48536 to 0.48865, saving model to weights.h5
217/217 - 1379s - loss: 0.6626 - auc_2: 0.4979 - binary_accuracy: 0.6007 - val_loss: 0.6672 - val_auc_2: 0.4886 - val_binary_accuracy: 0.5841 - 1379s/epoch - 6s/step
Epoch 3/7
